# Setup Athena Database

The dataset was downloaded locally from Hugging Face [here](https://huggingface.co/datasets/openfoodfacts/product-database/blob/main/food.parquet), then uploaded to our public S3 bucket.

In [1]:
# Install dependencies
!pip install --upgrade boto3 botocore awscli

  Using cached boto3-1.40.40-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.40.40-py3-none-any.whl.metadata (5.7 kB)
  Using cached awscli-1.42.40-py3-none-any.whl.metadata (11 kB)
  Using cached s3transfer-0.14.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached docutils-0.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached rsa-4.7.2-py3-none-any.whl.metadata (3.6 kB)
Using cached boto3-1.40.40-py3-none-any.whl (139 kB)
Using cached botocore-1.40.40-py3-none-any.whl (14.0 MB)
Using cached s3transfer-0.14.0-py3-none-any.whl (85 kB)
Using cached awscli-1.42.40-py3-none-any.whl (4.7 MB)
Using cached docutils-0.19-py3-none-any.whl (570 kB)
Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9.1
    Uninstalling rsa-4.9.1:
      Successfully uninstalled rsa-4.9.1
  Attempting uninstall: botocorem━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [docutils]
    Found existing installation: botocore 1.37.1━━━━━━━━━━━━━━ 1/6 [do

In [1]:
# Import libraries
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Setup boto and sagemaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
print('Boto and Sagemaker Sessions Initialized...\nBucket: {0}\nRegion: {1}'.format(bucket, region))

Boto and Sagemaker Sessions Initialized...
Bucket: sagemaker-us-east-1-654654380268
Region: us-east-1


## Verify Access to Dataset from Public S3 Bucket

In [19]:
# Verify the public S3 bucket contents
!aws s3 ls s3://aai-540-openfoodfacts/

2025-09-18 04:28:03 4253536290 food.parquet


In [4]:
# Set S3 source location (public S3 bucket)
s3_public_folder_path = "s3://aai-540-openfoodfacts"
s3_public_path_parquet = "s3://aai-540-openfoodfacts/food.parquet"
%store s3_public_folder_path
%store s3_public_path_parquet

Stored 's3_public_folder_path' (str)
Stored 's3_public_path_parquet' (str)


## Create Athena Database

In [5]:
# Initialize our database name
database_name = 'foodfacts'

# Set S3 staging directory
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# Create connection to Athena database
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [7]:
# Create our database
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS foodfacts


""


In [8]:
# Verify database was created
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,dsoaws
2,foodfacts
3,sagemaker_featurestore


## Create and Run a Glue Crawler for Raw Data
We created a glue crawler to automatically create a table of the dataset file.

In [9]:
# Initialize the AWS Glue client
glue_client = boto3.client('glue', region_name=region)

crawler_name = 'food_products_crawler'

# Create the Glue Crawler
try:
    glue_client.create_crawler(
        Name=crawler_name,
        Role=role,
        DatabaseName=database_name,
        Targets={'S3Targets': [{'Path': s3_public_folder_path}]},
    )
    print(f"Crawler '{crawler_name}' created successfully.")
except glue_client.exceptions.AlreadyExistsException:
    print(f"Crawler '{crawler_name}' already exists.")

Crawler 'food_products_crawler' already exists.


In [10]:
# Verify Creation of Glue Crawler
crawlers = glue_client.get_crawlers()
for crawler in crawlers['Crawlers']:
    print(f"Crawler Name: {crawler['Name']}, State: {crawler['State']}")

# Check if table was created from crawler
target_table = 'aai_540_openfoodfacts'
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
target_table_created = False

if target_table in df_show.iloc[:, 0].values:
    print(f"Table '{target_table}' exists!")
    target_table_created = True
else:
    print(f"Table '{target_table}' not found.")
    target_table_created = False

Crawler Name: food_products_crawler, State: READY
Table 'aai_540_openfoodfacts' exists!


In [11]:
# Start the Glue Crawler if target table not created
if not target_table:
    print(f"Starting crawler '{crawler_name}'...")
    glue_client.start_crawler(Name=crawler_name)
    
    # Wait for the crawler to finish
    while True:
        response = glue_client.get_crawler(Name=crawler_name)
        state = response['Crawler']['State']
    
        if state == 'READY':
            print("Crawler finished successfully.")
            break
        elif state == 'STOPPING':
            print("Crawler is stopping...")
        elif state == 'RUNNING':
            print("Crawler is running...")
        elif state == 'FAILED':
            print("Crawler failed.")
            break
        
        time.sleep(15) # Wait 15 seconds before checking the status again
else:
    print(f"Skipping Crawler. Table '{target_table}' exists!")

Skipping Crawler. Table 'aai_540_openfoodfacts' exists!


In [7]:
# Save table name
raw_table_name = "aai_540_openfoodfacts"
%store raw_table_name

Stored 'raw_table_name' (str)


## Verify Querying the Database

In [11]:
# Run Count query
statement = """SELECT COUNT(*) as count FROM {}.{}""".format(
    database_name, raw_table_name
)
print(statement)
df = pd.read_sql(statement, conn)
df

SELECT COUNT(*) as count FROM foodfacts.aai_540_openfoodfacts


,count
0,4029225


In [10]:
# Run sample query
statement = """SELECT * FROM {}.{} LIMIT 5""".format(
    database_name, raw_table_name
)
print(statement)
df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM foodfacts.aai_540_openfoodfacts LIMIT 5


,additives_n,additives_tags,allergens_tags,brands_tags,brands,categories,categories_tags,categories_properties,checkers_tags,ciqual_food_name_tags,...,states_tags,stores_tags,stores,traces_tags,unique_scans_n,unknown_ingredients_n,unknown_nutrients_tags,vitamins_tags,with_non_nutritive_sweeteners,with_sweeteners
0,NaN,None,[],None,None,None,None,"{'ciqual_food_code': None, 'agribalyse_food_co...",[],None,...,"[en:to-be-completed, en:nutrition-facts-comple...",None,None,[],NaN,NaN,[],None,None,None
1,NaN,None,[],[nice-rice],Nice rice,None,None,"{'ciqual_food_code': None, 'agribalyse_food_co...",[],None,...,"[en:to-be-completed, en:nutrition-facts-comple...",None,None,[],NaN,NaN,[],None,None,None
2,NaN,None,[],[delifin],Delifin,None,None,"{'ciqual_food_code': None, 'agribalyse_food_co...",[],None,...,"[en:to-be-completed, en:nutrition-facts-comple...",None,None,[],NaN,NaN,[],None,None,None
3,NaN,[],[],[pickwick],Pickwick,"Snacks,Sweet snacks,Biscuits and cakes,Biscuits","[en:snacks, en:sweet-snacks, en:biscuits-and-c...","{'ciqual_food_code': None, 'agribalyse_food_co...",[],None,...,"[en:to-be-completed, en:nutrition-facts-to-be-...",[],,[],NaN,NaN,[],[],None,None
4,0.0,[],[],[xx:carmen-s],Carmen’s,None,None,"{'ciqual_food_code': None, 'agribalyse_food_co...",[],None,...,"[en:to-be-completed, en:nutrition-facts-comple...",None,None,[],3.0,0.0,[],[en:vitamin-e],None,None


## Create a Filtered Subset Table

In [21]:
# Filter down dataset to food products in United States with selected raw columns
us_table_name = 'food_us_100k'
subset_path = 'food_us_subset_100k/'
full_s3_path = f's3://{bucket}/{subset_path}'

statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (
  format = 'PARQUET',
  parquet_compression = 'SNAPPY',
  external_location = '{}'
) AS
SELECT
  code,
  product_name,
  nutriments,
  nova_group,
  additives_n,
  ingredients_n,
  nutriscore_score
FROM
  {}.{}
WHERE
  contains(countries_tags, 'en:united-states')
  AND nutriscore_score IS NOT NULL 
  AND code IS NOT NULL 
  AND product_name IS NOT NULL 
  AND nova_group IS NOT NULL
  AND additives_n IS NOT NULL
  AND ingredients_n IS NOT NULL
  ORDER BY random()
LIMIT 100000""".format(
    database_name, us_table_name, full_s3_path, database_name, raw_table_name
)
print(statement)
pd.read_sql(statement, conn)

CREATE TABLE IF NOT EXISTS foodfacts.food_us_100k
WITH (
  format = 'PARQUET',
  parquet_compression = 'SNAPPY',
  external_location = 's3://sagemaker-us-east-1-654654380268/food_us_subset_100k/'
) AS
SELECT
  code,
  product_name,
  nutriments,
  nova_group,
  additives_n,
  ingredients_n,
  nutriscore_score
FROM
  foodfacts.aai_540_openfoodfacts
WHERE
  contains(countries_tags, 'en:united-states')
  AND nutriscore_score IS NOT NULL 
  AND code IS NOT NULL 
  AND product_name IS NOT NULL 
  AND nova_group IS NOT NULL
  AND additives_n IS NOT NULL
  AND ingredients_n IS NOT NULL
  ORDER BY random()
LIMIT 100000


,rows


In [22]:
# Run Count query on filtered subset
statement = """SELECT COUNT(*) AS COUNT FROM {}.{}""".format(
    database_name, us_table_name
)
print(statement)
df = pd.read_sql(statement, conn)
df

SELECT COUNT(*) AS COUNT FROM foodfacts.food_us_100k


,COUNT
0,100000


In [23]:
# View rows from subset data
statement = """SELECT * FROM {}.{} LIMIT 5;""".format(
    database_name, us_table_name
)
print(statement)
df_us = pd.read_sql(statement, conn)
df_us

SELECT * FROM foodfacts.food_us_100k LIMIT 5;


,code,product_name,nutriments,nova_group,additives_n,ingredients_n,nutriscore_score
0,0761706140406,"[{'lang': 'main', 'text': 'Bubble gums balls'}...","[{'name': 'calcium', 'value': 0.0, '100g': 0.0...",4,10,20,18
1,0782993057260,"[{'lang': 'main', 'text': 'Organic premium her...","[{'name': 'proteins', 'value': 0.0, '100g': 0....",1,0,1,0
2,0897064001049,"[{'lang': 'main', 'text': 'Nine sons rising'},...","[{'name': 'proteins', 'value': 0.0, '100g': 0....",3,0,4,15
3,0719212799700,"[{'lang': 'main', 'text': 'Chile con queso med...","[{'name': 'saturated-fat', 'value': 3.03, '100...",4,4,34,9
4,0713733083386,"[{'lang': 'main', 'text': 'Taco shells'}, {'la...","[{'name': 'energy-kcal', 'value': 455.0, '100g...",4,7,24,15
